### Задание 6.1
1/1 point (graded)

Предлагаем использовать датасет, с которым вы работали при создании пайплайна (файл <a href="https://lms.skillfactory.ru/assets/courseware/v1/4ac05d2d2721ace4a495c186925a350b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/Red.zip">Red.csv</a> ).

Вам следует выполнить следующее:

* Добавить обработку столбца 'Region' в пайплайн, полученный ранее в модуле, с использованием OrdinalEncoder.

Важно! Для совпадения результатов процесс трансформации столбцов должен выполняться в следующей последовательности:

1. Кодирование столбца 'Region'.
2. Стандартизация столбца 'Price'.
3. Кодирование столбца 'Country'.

* Обучить на тренировочном наборе данных пайплайн и оценить качество модели по метрике RMSE на тестовом наборе (файл <a href="https://lms.skillfactory.ru/assets/courseware/v1/27d0c9cfaebda5e496a455e8dc84a86a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/Red_test.zip">Red_test.csv</a> ).
* Зафиксировать random_state=42.
* Сохранить пайплайн в файл pipeline_wine.pkl.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [44]:
#загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
import joblib

In [38]:
def rmse(y_true, y_pred):
    print(
    f'Качество модели по RMSE: {mean_squared_error(y_true, y_pred, squared=False):.4f}')

In [39]:
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))
print(ct)

pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))])
print(pipeline)

df_wine = pd.read_csv('./data/Red.zip')
X_train = df_wine[['Region', 'Price', 'Country']]
y_train = df_wine['Rating']

pipeline.fit(X_train, y_train)

df_wine_test = pd.read_csv('./data/Red_test.zip')
X_test = df_wine_test[['Region', 'Price', 'Country']]
y_test = df_wine_test['Rating']

y_test_pred = pipeline.predict(X_test)
rmse(y_test, y_test_pred)

joblib.dump(pipeline, 'pipepline_wine.pkl')

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])
Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(n_jobs=-1, random_state=42))])
Качество модели по RMSE: 0.0765


['pipepline_wine.pkl']

### Задание 6.2
1 point possible (graded)

Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [40]:
pipeline.set_params(rf__n_estimators=200)
print(pipeline)
pipeline.fit(X_train, y_train)
y_test_pred = pipeline.predict(X_test)
rmse(y_test, y_test_pred)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, n_jobs=-1,
                                       random_state=42))])
Качество модели по RMSE: 0.0761


### Задание 6.3
1 point possible (graded)

Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

* Собрать StackingRegressor:
1. В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
2. В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
3. Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).

* Зафиксировать random_state=42 (для всех моделей).
* Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
* Обучить модель на тренировочной выборке.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округлённый до второго знака после запятой.

In [45]:
# Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('rcv', RidgeCV()),
    ('lr',  DecisionTreeRegressor(random_state=42))
]

# Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(
        n_estimators=10,
        random_state=42,
        n_jobs=-1),
    n_jobs=-1
)

pipeline = Pipeline(
    [('ct', ct), ('sr', reg)])
print(pipeline)

pipeline.fit(X_train, y_train)

y_test_pred = pipeline.predict(X_test)
rmse(y_test, y_test_pred)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('sr',
                 StackingRegressor(estimators=[('rcv', RidgeCV()),
                                               ('lr',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         n_jobs=-1,
                     